In [192]:
#import dependencies

import pandas as pd     
import matplotlib.pyplot as plt

In [193]:
#read in csvs

electiondata = pd.read_csv("1976-2016-president.csv")
rate1 = pd.read_csv("1980-2014 November General Election - Turnout Rates.csv")
rate2 = pd.read_csv("2016 November General Election - Turnout Rates.csv")
electoralvotes = pd.read_csv("Available Electoral votes .csv")
parties = pd.read_csv("parties.csv")

In [194]:
electiondata

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,2016,Wyoming,WY,56,83,68,US President,NaN,NaN,True,6904,258788,20171015,NaN
3736,2016,Wyoming,WY,56,83,68,US President,NaN,independent,False,3224,258788,20171015,NaN
3737,2016,Wyoming,WY,56,83,68,US President,Blank Vote,NaN,False,2661,258788,20171015,NaN
3738,2016,Wyoming,WY,56,83,68,US President,"Castle, Darrell L.",constitution party,False,2042,258788,20171015,NaN


In [195]:
allparties = electiondata[["candidate", "party"]]
allparties.drop_duplicates(inplace=True)
allparties
allparties.to_csv("allparties.csv", index=False)

In [196]:
#clean up voting data 

#file one
rate1.rename(columns={"Unnamed: 0": "year", "Denominators" : "Eligible_voting_pop"}, inplace= True)
voting_data_1 = rate1[["year", "State", "Eligible_voting_pop"]]
voting_data_1 = voting_data_1.drop(voting_data_1.index[0])

#file two
rate2.rename(columns={"Denominators" : "Eligible_voting_pop"}, inplace= True)
rate2["year"] = '2016'
voting_data_2 = rate2[["year", "State", "Eligible_voting_pop"]]
voting_data_2 = voting_data_2.drop(voting_data_2.index[0])

In [197]:
#merge dataframes

total_voting = voting_data_2.append(voting_data_1)

In [198]:
#check dtypes on dataframes

#total_voting.dtypes
    #year is object

#electiondata.dtypes
    #year is int64

In [199]:
#convert year to int for merge

total_voting['year'] = total_voting.year.astype(str).astype(int)

In [200]:
#merge dataframes 

merged_df = electiondata.merge(total_voting, how='left', left_on=['state','year'], right_on=["State","year"])
merged_df

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes,State,Eligible_voting_pop
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN,NaN,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN,NaN,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN,NaN,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN,NaN,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,2016,Wyoming,WY,56,83,68,US President,NaN,NaN,True,6904,258788,20171015,NaN,Wyoming,"429,682"
3736,2016,Wyoming,WY,56,83,68,US President,NaN,independent,False,3224,258788,20171015,NaN,Wyoming,"429,682"
3737,2016,Wyoming,WY,56,83,68,US President,Blank Vote,NaN,False,2661,258788,20171015,NaN,Wyoming,"429,682"
3738,2016,Wyoming,WY,56,83,68,US President,"Castle, Darrell L.",constitution party,False,2042,258788,20171015,NaN,Wyoming,"429,682"


In [201]:
#merge electoral college votes dataframe and drop columns

working_df = merged_df.merge(electoralvotes, how='left', left_on=['state','year'], right_on=["State","Year"])
final_working_df = working_df.drop(['state_po', 'state_fips', "state_cen", "state_ic", "office", "version", "notes", "State ID", "State_x","State_y", "Year", "writein"], axis=1)

#create key for merge
final_working_df["new"] = final_working_df["candidate"] + final_working_df["year"].astype(str)

final_working_df.replace("Mitt, Romney", "Romney, Mitt", inplace=True)
final_working_df.replace("Hedges, Jim", "Hedges, James", inplace=True)
final_working_df.replace("Haylard, Helen", "Halyard, Helen", inplace=True)
final_working_df.replace("Stevens, Thomas Robert ""Tom""", "Stevens, Thomas R.", inplace=True)
final_working_df.replace("White, Jerry", "White, Jerome ""Jerry""", inplace=True)


final_working_df["candidate"] = final_working_df['candidate'].fillna("unknown")
final_working_df["party"] = final_working_df['party'].fillna("unknown")
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].fillna(.00000000000000000000000000000000000000000000000001)


In [202]:
final_working_df = final_working_df.merge(parties,how= "left", left_on= "candidate", right_on="name")
final_working_df

,year,state,candidate,party_x,candidatevotes,totalvotes,Eligible_voting_pop,electoralvotes,new,name,party_y
0,1976,Alabama,"Carter, Jimmy",democrat,659170,1182850,1e-50,9.0,"Carter, Jimmy1976","Carter, Jimmy",democrat
1,1976,Alabama,"Ford, Gerald",republican,504070,1182850,1e-50,9.0,"Ford, Gerald1976","Ford, Gerald",republican
2,1976,Alabama,"Maddox, Lester",american independent party,9198,1182850,1e-50,9.0,"Maddox, Lester1976","Maddox, Lester",constitution party
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",prohibition,6669,1182850,1e-50,9.0,"Bubar, Benjamin """"Ben""""1976","Bubar, Benjamin """"Ben""""",prohibition
4,1976,Alabama,"Hall, Gus",communist party use,1954,1182850,1e-50,9.0,"Hall, Gus1976","Hall, Gus",independent
...,...,...,...,...,...,...,...,...,...,...,...
3735,2016,Wyoming,unknown,unknown,6904,258788,"429,682",3.0,NaN,NaN,NaN
3736,2016,Wyoming,unknown,independent,3224,258788,"429,682",3.0,NaN,NaN,NaN
3737,2016,Wyoming,Blank Vote,unknown,2661,258788,"429,682",3.0,Blank Vote2016,NaN,NaN
3738,2016,Wyoming,"Castle, Darrell L.",constitution party,2042,258788,"429,682",3.0,"Castle, Darrell L.2016","Castle, Darrell L.",constitution party


In [203]:
final_working_df["party_y"] = final_working_df['party_y'].fillna(0)

for row in final_working_df["party_y"]:
    final_working_df.loc[final_working_df["party_y"] != 0, ["party_x"]] = final_working_df["party_y"]

In [204]:
final_working_df

,year,state,candidate,party_x,candidatevotes,totalvotes,Eligible_voting_pop,electoralvotes,new,name,party_y
0,1976,Alabama,"Carter, Jimmy",democrat,659170,1182850,1e-50,9.0,"Carter, Jimmy1976","Carter, Jimmy",democrat
1,1976,Alabama,"Ford, Gerald",republican,504070,1182850,1e-50,9.0,"Ford, Gerald1976","Ford, Gerald",republican
2,1976,Alabama,"Maddox, Lester",constitution party,9198,1182850,1e-50,9.0,"Maddox, Lester1976","Maddox, Lester",constitution party
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",prohibition,6669,1182850,1e-50,9.0,"Bubar, Benjamin """"Ben""""1976","Bubar, Benjamin """"Ben""""",prohibition
4,1976,Alabama,"Hall, Gus",independent,1954,1182850,1e-50,9.0,"Hall, Gus1976","Hall, Gus",independent
...,...,...,...,...,...,...,...,...,...,...,...
3735,2016,Wyoming,unknown,unknown,6904,258788,"429,682",3.0,NaN,NaN,0
3736,2016,Wyoming,unknown,independent,3224,258788,"429,682",3.0,NaN,NaN,0
3737,2016,Wyoming,Blank Vote,unknown,2661,258788,"429,682",3.0,Blank Vote2016,NaN,0
3738,2016,Wyoming,"Castle, Darrell L.",constitution party,2042,258788,"429,682",3.0,"Castle, Darrell L.2016","Castle, Darrell L.",constitution party


In [205]:
#you need to make a seperate dataframe with candidate and party, do the group by without party, and then merge em back

party_df = final_working_df[["candidate", "party_x"]]

final_working_df = final_working_df[["year", "state", "candidate", "totalvotes", "Eligible_voting_pop", "new", "electoralvotes", "candidatevotes"]]

#final_working_df = final_working_df.loc[final_working_df["candidate"] == 'Bubar, Benjamin ""Ben""']
final_working_df

,year,state,candidate,totalvotes,Eligible_voting_pop,new,electoralvotes,candidatevotes
0,1976,Alabama,"Carter, Jimmy",1182850,1e-50,"Carter, Jimmy1976",9.0,659170
1,1976,Alabama,"Ford, Gerald",1182850,1e-50,"Ford, Gerald1976",9.0,504070
2,1976,Alabama,"Maddox, Lester",1182850,1e-50,"Maddox, Lester1976",9.0,9198
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669
4,1976,Alabama,"Hall, Gus",1182850,1e-50,"Hall, Gus1976",9.0,1954
...,...,...,...,...,...,...,...,...
3735,2016,Wyoming,unknown,258788,"429,682",NaN,3.0,6904
3736,2016,Wyoming,unknown,258788,"429,682",NaN,3.0,3224
3737,2016,Wyoming,Blank Vote,258788,"429,682",Blank Vote2016,3.0,2661
3738,2016,Wyoming,"Castle, Darrell L.",258788,"429,682","Castle, Darrell L.2016",3.0,2042


In [206]:
final_working_df = final_working_df.groupby(["year", "state", "candidate", "totalvotes", "Eligible_voting_pop", "new", "electoralvotes"]).sum().reset_index()
final_working_df


,year,state,candidate,totalvotes,Eligible_voting_pop,new,electoralvotes,candidatevotes
0,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669
1,1976,Alabama,"Carter, Jimmy",1182850,1e-50,"Carter, Jimmy1976",9.0,659170
2,1976,Alabama,"Ford, Gerald",1182850,1e-50,"Ford, Gerald1976",9.0,504070
3,1976,Alabama,"Hall, Gus",1182850,1e-50,"Hall, Gus1976",9.0,1954
4,1976,Alabama,"Macbride, Roger",1182850,1e-50,"Macbride, Roger1976",9.0,1481
...,...,...,...,...,...,...,...,...
3355,2016,Wyoming,"Castle, Darrell L.",258788,"429,682","Castle, Darrell L.2016",3.0,2042
3356,2016,Wyoming,"Clinton, Hillary",258788,"429,682","Clinton, Hillary2016",3.0,55973
3357,2016,Wyoming,"Johnson, Gary",258788,"429,682","Johnson, Gary2016",3.0,13287
3358,2016,Wyoming,Over Vote,258788,"429,682",Over Vote2016,3.0,278


In [207]:
final_working_df = final_working_df.merge(party_df, how="left" , left_on = 'candidate', right_on= "candidate")
final_working_df

,year,state,candidate,totalvotes,Eligible_voting_pop,new,electoralvotes,candidatevotes,party_x
0,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
1,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
2,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
4,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
...,...,...,...,...,...,...,...,...,...
211562,2016,Wyoming,"Trump, Donald J.",258788,"429,682","Trump, Donald J.2016",3.0,174419,republican
211563,2016,Wyoming,"Trump, Donald J.",258788,"429,682","Trump, Donald J.2016",3.0,174419,republican
211564,2016,Wyoming,"Trump, Donald J.",258788,"429,682","Trump, Donald J.2016",3.0,174419,republican
211565,2016,Wyoming,"Trump, Donald J.",258788,"429,682","Trump, Donald J.2016",3.0,174419,republican


In [208]:

final_working_df.rename(columns={"party_x" : "party"}, inplace=True)
final_working_df = final_working_df.drop_duplicates()


final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].replace({'\$': '', ',': ''}, regex=True)
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].astype(str).astype(float)
final_working_df

,year,state,candidate,totalvotes,Eligible_voting_pop,new,electoralvotes,candidatevotes,party
0,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1.000000e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition
19,1976,Alabama,"Carter, Jimmy",1182850,1.000000e-50,"Carter, Jimmy1976",9.0,659170,democrat
122,1976,Alabama,"Ford, Gerald",1182850,1.000000e-50,"Ford, Gerald1976",9.0,504070,republican
174,1976,Alabama,"Hall, Gus",1182850,1.000000e-50,"Hall, Gus1976",9.0,1954,independent
241,1976,Alabama,"Macbride, Roger",1182850,1.000000e-50,"Macbride, Roger1976",9.0,1481,libertarian
...,...,...,...,...,...,...,...,...,...
211332,2016,Wyoming,"Castle, Darrell L.",258788,4.296820e+05,"Castle, Darrell L.2016",3.0,2042,constitution party
211356,2016,Wyoming,"Clinton, Hillary",258788,4.296820e+05,"Clinton, Hillary2016",3.0,55973,democrat
211410,2016,Wyoming,"Johnson, Gary",258788,4.296820e+05,"Johnson, Gary2016",3.0,13287,libertarian
211506,2016,Wyoming,Over Vote,258788,4.296820e+05,Over Vote2016,3.0,278,unknown


In [209]:
#find top candidates by popular vote

find_top_candidates_df = final_working_df[["year","candidate", "candidatevotes"]]

find_top_candidates_df = find_top_candidates_df.groupby(["candidate", "year"]).sum().reset_index()

find_top_candidates_df= find_top_candidates_df.sort_values(by=["year", "candidatevotes"], ascending=False).groupby("year").head(2)

top_candidates_df = pd.DataFrame(find_top_candidates_df)

#create key for merge
top_candidates_df["new"] = top_candidates_df["candidate"] + top_candidates_df["year"].astype(str)

#create dataframe with only key for merge
#loop through to deduplicate 
only_winners = top_candidates_df[["new"]]

toppy = []
for row in only_winners["new"]:
    if row not in toppy:
        toppy.append(row)

toppy=pd.DataFrame(toppy)


find_top_candidates_df 

,candidate,year,candidatevotes,new
55,"Clinton, Hillary",2016,65570751,"Clinton, Hillary2016"
232,"Trump, Donald J.",2016,62972339,"Trump, Donald J.2016"
178,"Obama, Barack H.",2012,65632590,"Obama, Barack H.2012"
210,"Romney, Mitt",2012,60910771,"Romney, Mitt2012"
177,"Obama, Barack H.",2008,69252659,"Obama, Barack H.2008"
149,"McCain, John",2008,59930916,"McCain, John2008"
43,"Bush, George W.",2004,62007029,"Bush, George W.2004"
123,"Kerry, John",2004,58825109,"Kerry, John2004"
86,"Gore, Al",2000,50824139,"Gore, Al2000"
42,"Bush, George W.",2000,50438096,"Bush, George W.2000"


In [210]:
#merge to create column for flag

final_final_but_not_quite = final_working_df.merge(toppy, how="left", left_on= "new", right_on= 0)

#fill na in new columnt with 0
final_final_but_not_quite[0] = final_final_but_not_quite[0].fillna(0)

#fill na in eligible voting pop for math


final_final_but_not_quite

,year,state,candidate,totalvotes,Eligible_voting_pop,new,electoralvotes,candidatevotes,party,0
0,1976,Alabama,"Bubar, Benjamin """"Ben""""",1182850,1.000000e-50,"Bubar, Benjamin """"Ben""""1976",9.0,6669,prohibition,0
1,1976,Alabama,"Carter, Jimmy",1182850,1.000000e-50,"Carter, Jimmy1976",9.0,659170,democrat,"Carter, Jimmy1976"
2,1976,Alabama,"Ford, Gerald",1182850,1.000000e-50,"Ford, Gerald1976",9.0,504070,republican,"Ford, Gerald1976"
3,1976,Alabama,"Hall, Gus",1182850,1.000000e-50,"Hall, Gus1976",9.0,1954,independent,0
4,1976,Alabama,"Macbride, Roger",1182850,1.000000e-50,"Macbride, Roger1976",9.0,1481,libertarian,0
...,...,...,...,...,...,...,...,...,...,...
4391,2016,Wyoming,"Castle, Darrell L.",258788,4.296820e+05,"Castle, Darrell L.2016",3.0,2042,constitution party,0
4392,2016,Wyoming,"Clinton, Hillary",258788,4.296820e+05,"Clinton, Hillary2016",3.0,55973,democrat,"Clinton, Hillary2016"
4393,2016,Wyoming,"Johnson, Gary",258788,4.296820e+05,"Johnson, Gary2016",3.0,13287,libertarian,0
4394,2016,Wyoming,Over Vote,258788,4.296820e+05,Over Vote2016,3.0,278,unknown,0


In [211]:
#using flag, change candidate name to other if not in top 2

final_final_but_not_quite.loc[final_final_but_not_quite[0] == 0, ["candidate"]] = 'Other'

#drop flag column
maybe_final = final_final_but_not_quite.drop([0], axis=1)

#do math for % won
for row in maybe_final:
    maybe_final["perc_won"] = maybe_final["candidatevotes"] / maybe_final["totalvotes"] 

#maybe_final["Eligible_voting_pop"] = maybe_final["Eligible_voting_pop"].replace({'\$': '', ',': ''}, regex=True)
#maybe_final["Eligible_voting_pop"] = maybe_final["Eligible_voting_pop"].astype(str).astype(float)
#maybe_final.dtypes


#for row in maybe_final:
 #   maybe_final["perc_participated"] = maybe_final["candidatevotes"] / maybe_final["Eligible_voting_pop"] * 100


#maybe_final = maybe_final.loc[maybe_final["state"] == "New York"]
#maybe_final = maybe_final.loc[maybe_final["year"] == 2000]
maybe_final.dtypes

year                     int64
state                   object
candidate               object
totalvotes               int64
Eligible_voting_pop    float64
new                     object
electoralvotes         float64
candidatevotes           int64
party                   object
perc_won               float64
dtype: object

In [212]:
find_top_candidates_state_df = maybe_final[["year","state","candidate", "candidatevotes", "new"]]

#find_top_candidates_state_df.drop_duplicates(keep="first", inplace=True)

#find_top_candidates_state_df = find_top_candidates_state_df.loc[find_top_candidates_state_df["state"] == "New York"]
#ind_top_candidates_state_df = find_top_candidates_state_df.loc[find_top_candidates_state_df["year"] == 2000]
find_top_candidates_state_df

,year,state,candidate,candidatevotes,new
0,1976,Alabama,Other,6669,"Bubar, Benjamin """"Ben""""1976"
1,1976,Alabama,"Carter, Jimmy",659170,"Carter, Jimmy1976"
2,1976,Alabama,"Ford, Gerald",504070,"Ford, Gerald1976"
3,1976,Alabama,Other,1954,"Hall, Gus1976"
4,1976,Alabama,Other,1481,"Macbride, Roger1976"
...,...,...,...,...,...
4391,2016,Wyoming,Other,2042,"Castle, Darrell L.2016"
4392,2016,Wyoming,"Clinton, Hillary",55973,"Clinton, Hillary2016"
4393,2016,Wyoming,Other,13287,"Johnson, Gary2016"
4394,2016,Wyoming,Other,278,Over Vote2016


In [213]:
#sum candidate votes

find_top_candidates_state_df = find_top_candidates_state_df.groupby(["candidate", "year", "state"]).sum().reset_index()

#sort to get top candidate
find_top_candidates_state_df= find_top_candidates_state_df.sort_values(by=[ "year","candidatevotes", "state"], ascending=False)
find_top_candidates_state_df = find_top_candidates_state_df.sort_values(["year", "state"]).groupby(["state","year"]).head(1)

#create key for merge
find_top_candidates_state_df["new"] = find_top_candidates_state_df["candidate"] + find_top_candidates_state_df["year"].astype(str)

find_top_candidates_state_df

#find_top_candidates_state_df.loc[find_top_candidates_state_df["state"] == "New York"]

,candidate,year,state,candidatevotes,new
200,"Carter, Jimmy",1976,Alabama,659170,"Carter, Jimmy1976"
551,"Ford, Gerald",1976,Alaska,71555,"Ford, Gerald1976"
552,"Ford, Gerald",1976,Arizona,418642,"Ford, Gerald1976"
203,"Carter, Jimmy",1976,Arkansas,498604,"Carter, Jimmy1976"
554,"Ford, Gerald",1976,California,3882244,"Ford, Gerald1976"
...,...,...,...,...,...
445,"Clinton, Hillary",2016,Virginia,1981473,"Clinton, Hillary2016"
446,"Clinton, Hillary",2016,Washington,1742718,"Clinton, Hillary2016"
1627,"Trump, Donald J.",2016,West Virginia,489371,"Trump, Donald J.2016"
1628,"Trump, Donald J.",2016,Wisconsin,1405284,"Trump, Donald J.2016"


In [214]:
#merge dataframed for the state won flag

state_winner_added = maybe_final.merge(find_top_candidates_state_df, how="left", on= ["new", "state"], suffixes=('', 'drop'))

#fill na in flag column for loc and change
state_winner_added["candidatedrop"] = state_winner_added["candidatedrop"].fillna(0)

#loc and change values
state_winner_added.loc[state_winner_added["candidatedrop"] != 0, ["candidatedrop"]] = '1'
state_winner_added.loc[state_winner_added["candidatedrop"] == 0, ["candidatedrop"]] = '0'

#drop and rename columns
state_winner_added = state_winner_added.drop(["yeardrop",  "candidatevotesdrop", "new"], axis=1).rename(columns={"candidatedrop": "state_won_flag"})

#fill in na parties with unknown
state_winner_added["party"] = state_winner_added["party"].fillna("Unknown")

final_df = state_winner_added

final_df

,year,state,candidate,totalvotes,Eligible_voting_pop,electoralvotes,candidatevotes,party,perc_won,state_won_flag
0,1976,Alabama,Other,1182850,1.000000e-50,9.0,6669,prohibition,0.005638,0
1,1976,Alabama,"Carter, Jimmy",1182850,1.000000e-50,9.0,659170,democrat,0.557273,1
2,1976,Alabama,"Ford, Gerald",1182850,1.000000e-50,9.0,504070,republican,0.426149,0
3,1976,Alabama,Other,1182850,1.000000e-50,9.0,1954,independent,0.001652,0
4,1976,Alabama,Other,1182850,1.000000e-50,9.0,1481,libertarian,0.001252,0
...,...,...,...,...,...,...,...,...,...,...
4391,2016,Wyoming,Other,258788,4.296820e+05,3.0,2042,constitution party,0.007891,0
4392,2016,Wyoming,"Clinton, Hillary",258788,4.296820e+05,3.0,55973,democrat,0.216289,0
4393,2016,Wyoming,Other,258788,4.296820e+05,3.0,13287,libertarian,0.051343,0
4394,2016,Wyoming,Other,258788,4.296820e+05,3.0,278,unknown,0.001074,0


In [215]:
#electoral college df
electoral_college = final_df[["year", "state", "candidate","electoralvotes", "state_won_flag"]]
electoral_college = electoral_college.loc[electoral_college["state_won_flag"] == '1']

electoral_college = electoral_college.drop(["state","state_won_flag" ], axis=1)
electoral_college = electoral_college.groupby(["year", "candidate"]).sum().reset_index()

electoral_college["new"] = electoral_college["candidate"] + electoral_college["year"].astype(str)
#electoral_college = electoral_college.sort_values(["year", "electoralvotes"], ascending=False)



electoral_college_win = electoral_college.groupby("year").head(2)
electoral_college_win



,year,candidate,electoralvotes,new
0,1976,"Carter, Jimmy",294.0,"Carter, Jimmy1976"
1,1976,"Ford, Gerald",241.0,"Ford, Gerald1976"
2,1980,"Carter, Jimmy",46.0,"Carter, Jimmy1980"
3,1980,"Reagan, Ronald",489.0,"Reagan, Ronald1980"
4,1984,"Mondale, Walter",10.0,"Mondale, Walter1984"
5,1984,"Reagan, Ronald",525.0,"Reagan, Ronald1984"
6,1988,"Bush, George H.W.",426.0,"Bush, George H.W.1988"
7,1988,"Dukakis, Michael",109.0,"Dukakis, Michael1988"
8,1992,"Bush, George H.W.",168.0,"Bush, George H.W.1992"
9,1992,"Clinton, Bill",367.0,"Clinton, Bill1992"


In [216]:
#start of year level dataset

year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(1)
popular_vote = year_df[["year", "candidate", "candidatevotes"]]
popular_vote["new"] = popular_vote["candidate"] + popular_vote["year"].astype(str)



year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(3)

vote = year_df[["year", "candidate", "candidatevotes"]]
vote["new"] = vote["candidate"] + vote["year"].astype(str)

vote = vote.merge(popular_vote, how="left", left_on="new", right_on="new", suffixes=('', 'drop'))
vote = vote.merge(electoral_college, how="left", left_on="new", right_on="new", suffixes=('', 'drop1'))

In [217]:
vote = vote.drop(["new", "yeardrop", "yeardrop1", "candidatevotesdrop", "candidatedrop1"], axis=1)
vote["electoralvotes"] = vote["electoralvotes"].fillna(0)
vote["candidatedrop"] = vote["candidatedrop"].fillna(0)

vote.loc[vote["candidatedrop"] != 0, ["candidatedrop"]] = '1'
#vote.loc[vote["candidatedrop"] == 0, ["candidatedrop"]] = '0'

vote.rename(columns={"candidatedrop": "won_pop_vote_flag"}, inplace=True)

vote

,year,candidate,candidatevotes,won_pop_vote_flag,electoralvotes
0,2016,"Clinton, Hillary",65570751,1,223.0
1,2016,"Trump, Donald J.",62972339,0,305.0
2,2016,Other,8802017,0,84.0
3,2012,"Obama, Barack H.",65632590,1,329.0
4,2012,"Romney, Mitt",59620101,0,206.0
5,2012,Other,3617228,0,0.0
6,2008,"Obama, Barack H.",69252659,1,361.0
7,2008,"McCain, John",59930916,0,174.0
8,2008,Other,2129671,0,0.0
9,2004,"Bush, George W.",62007029,1,286.0


In [221]:
turnout_votes = final_working_df[["year", "totalvotes"]]
turnout_eligible = final_working_df[["year", "Eligible_voting_pop"]]

turnout_votes.drop_duplicates(inplace=True)
#turnout_votes

turnout_eligible.drop_duplicates(inplace=True)
#turnout_eligible

turnout_votes = turnout_votes.groupby("year").sum()
#turnout_votes

turnout_eligible = turnout_eligible.groupby("year").sum().reset_index()

turnout = turnout_votes.merge(turnout_eligible, how = "left", on="year")
turnout

turnout["perc"] = turnout["totalvotes"]/ turnout["Eligible_voting_pop"] 
turnout

,year,totalvotes,Eligible_voting_pop,perc
0,1976,81432514,1.000000e-50,8.143251e+57
1,1980,86322962,1.574318e+08,5.483196e-01
2,1984,92443573,1.649467e+08,5.604453e-01
3,1988,91393948,1.706992e+08,5.354094e-01
4,1992,104372208,1.769542e+08,5.898262e-01
5,1996,96204092,1.836282e+08,5.239070e-01
6,2000,105392088,1.912131e+08,5.511762e-01
7,2004,122121864,1.993528e+08,6.125915e-01
8,2008,131153400,2.081971e+08,6.299484e-01
9,2012,128846233,2.171725e+08,5.932898e-01
